## <center>This Model will Attempt to Predict the Type of Bicycle Used from Trip Data</center>

In [81]:
# Eliminating Warnings
import warnings
warnings.filterwarnings('ignore')

In [82]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

In [83]:
# Set up data path
data_path = 'data'

In [84]:
# Load base data to dataframe
all_data_file = 'all_data.csv'
all_df = pd.read_csv(os.path.join(data_path, all_data_file))
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,11/28/16,Citibike from Work,0.81,NaN,7.0,0.12,6.94,Citibike
1,2/24/20,Wards Island,11.66,1.0,33.0,1.55,7.52,Specialized
2,8/10/19,Summer Streets,12.85,1.0,29.0,1.48,8.66,Specialized
3,8/4/18,Summer Streets,11.55,1.0,25.0,1.42,8.15,Specialized
4,12/2/16,Citibike to Work,1.17,NaN,11.0,0.18,6.38,Citibike
...,...,...,...,...,...,...,...,...
348,9/24/20,Wards Island x 4,28.23,3.0,6.0,3.10,9.11,Specialized
349,7/29/16,Citibike to Work,1.05,NaN,12.0,0.20,5.25,Citibike
350,10/25/18,CP ML,5.93,NaN,36.0,0.60,9.88,Specialized
351,4/7/20,CP BL x 2,13.98,1.0,43.0,1.72,8.14,Specialized


In [85]:
# Defining fields to use
columns = ['Miles', 'Duration', 'Speed','Type']
# target = ['Type']

In [86]:
# Create Dataframe with desired columns
bike_df = all_df.loc[:,columns].copy()
bike_df

,Miles,Duration,Speed,Type
0,0.81,0.12,6.94,Citibike
1,11.66,1.55,7.52,Specialized
2,12.85,1.48,8.66,Specialized
3,11.55,1.42,8.15,Specialized
4,1.17,0.18,6.38,Citibike
...,...,...,...,...
348,28.23,3.10,9.11,Specialized
349,1.05,0.20,5.25,Citibike
350,5.93,0.60,9.88,Specialized
351,13.98,1.72,8.14,Specialized


In [87]:
# Create Dataframe with desired columns
# Demonstrating different way to do it
#new2_df = all_df[['Miles', 'Duration', 'Speed', 'Type']]
#new2_df

In [88]:
# Splitting out Features and Target
y = bike_df["Type"]
X = bike_df.drop(columns="Type")

In [89]:
# Splitting out Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [90]:
# Checking training data shape
X_train.shape

(264, 3)

In [91]:
# Checking testing data shape
X_test.shape

(89, 3)

In [92]:
# Creating the Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

In [93]:
# Training the Model
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [94]:
# Create Predictions
y_pred = classifier.predict(X_test)

In [95]:
# Get accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9662921348314607


In [96]:
# See Predictions side-by-side
# Convert Predictions to DataFrame
pred_df = pd.DataFrame(y_pred)
pred_df

,0
0,Specialized
1,Citibike
2,Specialized
3,Specialized
4,Specialized
...,...
84,Citibike
85,Citibike
86,Specialized
87,Citibike


In [97]:
# Convert Tests to DataFrame
test_df = pd.DataFrame(y_test)
test_df

,Type
222,Specialized
295,Citibike
108,Specialized
141,Specialized
92,Specialized
...,...
309,Citibike
228,Citibike
54,Specialized
172,Citibike


In [98]:
# Resetting index on Test set
# Export to CSV fil
test_data_file = 'test_data.csv'
test_df.to_csv(os.path.join(data_path, test_data_file), index=False)

In [99]:
# Reimporting Test Data File
test2_df = pd.read_csv(os.path.join(data_path, test_data_file))
test2_df

,Unnamed: 0,Type
0,222,Specialized
1,295,Citibike
2,108,Specialized
3,141,Specialized
4,92,Specialized
...,...,...
84,309,Citibike
85,228,Citibike
86,54,Specialized
87,172,Citibike


In [77]:
# Combining the test and predition dataframes for comparision
# horizontal_stack = pd.concat([survey_sub, survey_sub_last10], axis=1)
test_pred_df = pd.concat([test2_df, pred_df], axis = 1)
test_pred_df

,Type,0
0,Specialized,Specialized
1,Citibike,Citibike
2,Specialized,Specialized
3,Specialized,Specialized
4,Specialized,Specialized
...,...,...
84,Citibike,Citibike
85,Citibike,Citibike
86,Specialized,Specialized
87,Citibike,Citibike


In [79]:
# Exporting Combined Test and Prediction Dataframe for examination
test_pred_file = "test_pred.csv"
test_pred_df.to_csv(os.path.join(data_path, test_pred_file))